# Remapping analysis with pairs of neurons during search and homing part of the trials

We get the IFR and the firing rate maps for the neurons in different condition. We then look at the IFR association and firing rate map similarity for pairs of neurons in a given condition.

* Do cells that fire together in one environment also fire together in the next environment?
* Do cells with similar firing rate maps in one condition also have similar maps in another?


In [1]:
%load_ext autoreload
%autoreload 2

%run ../setup_project.py
%run ../neuronAutopi.py

prepareSessionsForSpatialAnalysisProject(sSesList,myProject.sessionList,pose_file_extension = ".pose_kf.npy")
from scipy.stats import pearsonr

Project name: autopi_ca1
dataPath: /ext_drives/d80/Jazi_etal_2023/autopi_ca1
Reading /ext_drives/d80/Jazi_etal_2023/autopi_ca1/sessionList
We have 39 testing sessions in the list
See myProject and sSesList objects
Loading Animal_pose and Spike_train, sSes.ap and sSes.cg


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:09<00:00,  4.10it/s]

Loading ses.trial_table_simple as ses.trials
Create condition intervals in ses.intervalDict


In [26]:
for ses, sSes in tqdm(zip(myProject.sessionList,sSesList)):
    getSearchHomingIntervals(ses,sSes)

39it [00:00, 53.26it/s]


## Instantaneous Firing rate associations

We can look at pairs of cells and test if their firing associations (r values) are preserve or change across different conditions.
The function below calculates the firing associations of pairs of neurons in different conditions.
Later on we can compare IFR association across conditions.

In [27]:

def ifrAssociationsPerCondition(ses,sSes,
                                 conditions=['searchToLeverPath_light','homingFromLeavingLever_light',
                                             'searchToLeverPath_dark','homingFromLeavingLever_dark',
                                            'searchToLeverPath_light_1','searchToLeverPath_light_2',
                                             'searchToLeverPath_dark_1','searchToLeverPath_dark_2',
                                            'homingFromLeavingLever_light_1','homingFromLeavingLever_light_2',
                                             'homingFromLeavingLever_dark_1','homingFromLeavingLever_dark_2'],
                                prefix="ifrAsso_"):
    """
    Calculate the instantaneous firing rate associations of pairs of neurons in a sets of conditions
    
    Arguments: 
    ses: autopipy session
    sSes: spikeA session
    conditions: list of condition names you want to analyze
    prefix: prefix for the DataFrame column names
    
    Return:
    Pandas DataFrame with the IFR association for each condition. One row per pair
    """ 
    
    
    # we will store the results in a dictionary, one list of scores per condition
    res={}
    for cond in conditions:
        
        
        nInfo = len(cond.split("_"))
        if nInfo == 2:
            navPathType = cond.split("_")[0]
            light = cond.split("_")[1]
        if nInfo == 3:
            navPathType = cond.split("_")[0]
            light = cond.split("_")[1]
            light = light + "_" + cond.split("_")[2]
        
        inter = ses.intervalDict[navPathType+"_"+light]
        
        
        # set the intervals for each neuron and calculate the IFR
        for n in sSes.cg.neuron_list:
            n.spike_train.unset_intervals()
            n.spike_train.set_intervals(inter)
            n.spike_train.instantaneous_firing_rate(bin_size_sec = 0.100, sigma = 1,outside_interval_solution="remove")
        # calculate firing association for all pairs of neurons
        sSes.cg.make_pairs(pair_type="combinations")
        rs = np.empty(len(sSes.cg.pairs)) # to store the results
        for i, (j,k) in enumerate(sSes.cg.pairs):
            n1=sSes.cg.neuron_list[j]
            n2=sSes.cg.neuron_list[k]
            rs[i] = pearsonr(n1.spike_train.ifr[0].flatten(),n2.spike_train.ifr[0].flatten())[0]
        res[cond]=rs
            
    #create a DataFrame from the dictionary
    res = pd.DataFrame(res)
    res = res.add_prefix(prefix)
    res["id1"] = [ sSes.name+"_"+sSes.cg.neuron_list[i].name for i,j in sSes.cg.pairs]
    res["id2"] = [ sSes.name+"_"+sSes.cg.neuron_list[j].name for i,j in sSes.cg.pairs]
    
    res["session"] = sSes.name
    return res


In [28]:
ses,sSes = list(zip(myProject.sessionList,sSesList))[0]
res = ifrAssociationsPerCondition(ses,sSes)

In [29]:
res

,ifrAsso_searchToLeverPath_light,ifrAsso_homingFromLeavingLever_light,ifrAsso_searchToLeverPath_dark,ifrAsso_homingFromLeavingLever_dark,ifrAsso_searchToLeverPath_light_1,ifrAsso_searchToLeverPath_light_2,ifrAsso_searchToLeverPath_dark_1,ifrAsso_searchToLeverPath_dark_2,ifrAsso_homingFromLeavingLever_light_1,ifrAsso_homingFromLeavingLever_light_2,ifrAsso_homingFromLeavingLever_dark_1,ifrAsso_homingFromLeavingLever_dark_2,id1,id2,session
0,-0.094626,-0.056125,0.118534,0.074950,-0.059297,-0.110526,0.120392,0.114425,0.007026,-0.095404,0.055488,0.069088,mn5824-20112020-0107_80,mn5824-20112020-0107_90,mn5824-20112020-0107
1,-0.047017,-0.048184,-0.161404,0.202896,0.009722,-0.092172,-0.178946,-0.140834,0.048399,-0.142756,0.257524,0.152992,mn5824-20112020-0107_80,mn5824-20112020-0107_92,mn5824-20112020-0107
2,-0.068192,-0.071393,-0.002655,0.148954,-0.049061,-0.136602,-0.001497,0.002332,-0.166136,-0.016337,0.104987,0.202446,mn5824-20112020-0107_80,mn5824-20112020-0107_96,mn5824-20112020-0107
3,0.013066,0.066866,-0.087000,-0.094489,-0.043636,0.112939,-0.101051,-0.075027,0.034606,0.097045,-0.065938,-0.106954,mn5824-20112020-0107_80,mn5824-20112020-0107_98,mn5824-20112020-0107
4,-0.005240,-0.077595,-0.113553,-0.161970,0.089546,-0.091446,-0.127471,-0.103601,-0.093201,-0.041316,-0.203758,-0.117246,mn5824-20112020-0107_80,mn5824-20112020-0107_100,mn5824-20112020-0107
5,-0.062487,-0.071766,0.243753,-0.060560,-0.048184,-0.087475,0.287198,0.204939,-0.086572,-0.080460,-0.031996,-0.075731,mn5824-20112020-0107_80,mn5824-20112020-0107_114,mn5824-20112020-0107
6,0.069730,-0.019774,-0.060163,0.007369,0.112209,0.049321,-0.100972,-0.046457,0.011921,-0.077096,-0.013715,0.025926,mn5824-20112020-0107_80,mn5824-20112020-0107_116,mn5824-20112020-0107
7,0.019182,0.052472,0.026715,0.014809,0.034000,-0.034228,0.026742,0.028670,0.066728,0.035795,0.085332,-0.015906,mn5824-20112020-0107_80,mn5824-20112020-0107_118,mn5824-20112020-0107
8,0.050511,0.019300,-0.141930,-0.098960,0.027101,0.080536,-0.156205,-0.132270,0.094087,-0.042018,-0.040299,-0.127969,mn5824-20112020-0107_80,mn5824-20112020-0107_122,mn5824-20112020-0107
9,-0.011087,-0.034362,-0.017883,0.022247,NaN,-0.018148,-0.006112,-0.029195,NaN,-0.056230,-0.003283,0.063743,mn5824-20112020-0107_80,mn5824-20112020-0107_124,mn5824-20112020-0107


In [30]:
import warnings

warnings.filterwarnings("ignore", category=stats.ConstantInputWarning)

In [31]:
res_ifr=pd.concat([ifrAssociationsPerCondition(ses,sSes) for ses,sSes in tqdm(zip(myProject.sessionList,sSesList))],ignore_index=True)

39it [03:53,  5.98s/it]


## Firing rate map similarities

As a complementary analysis to the IFR association, we can also look at firing rate map similarity in different conditions. 


In [32]:
a = 'searchToLeverPath_light_1'
a.split("_")

['searchToLeverPath', 'light', '1']

In [9]:
def map_cor(a,b):
    """
    Correlation coefficient between two firing rate maps
    
    Arguments:
    a: 2D np.array (map1)
    b: 2D np.array (map2)
    
    Returns:
    Pearson correlation coefficient between a and b
    """
    a = a.flatten()
    b = b.flatten()
    indices = np.logical_and(~np.isnan(a), ~np.isnan(b))
    r,p = pearsonr(a[indices],b[indices])
    return r

In [10]:
def mapSimilarityPerCondition(ses,sSes, onlyArena=True,
                              conditions=['searchToLeverPath_light','homingFromLeavingLever_light','searchToLeverPath_dark','homingFromLeavingLever_dark',
                                          'searchToLeverPath_light_1', 'searchToLeverPath_light_2', 'searchToLeverPath_dark_1', 'searchToLeverPath_dark_2',
                                          'homingFromLeavingLever_light_1', 'homingFromLeavingLever_light_2', 'homingFromLeavingLever_dark_1', 'homingFromLeavingLever_dark_2'],
                              prefix="mapSim_"):
    """
    Calculate the map similarity of pairs of neurons in a sets of conditions
    
    Arguments: 
    ses: autopipy session
    sSes: spikeA session
    conditions: list of condition names you want to analyze
    prefix: prefix for the DataFrame column names
    
    Return:
    Pandas DataFrame with map similarity for each condition. One row per pair
    """ 
    # we will store the results in a dictionary, one list of scores per condition
    
    
    res={}
    xy_range=np.array([[-50,-90],[50,60]])
    
    for cond in conditions:
        # set the intervals for each neuron and animal pose and calculate the maps
        
        nInfo = len(cond.split("_"))
        if nInfo == 2:
            navPathType = cond.split("_")[0]
            light = cond.split("_")[1]
        if nInfo == 3:
            navPathType = cond.split("_")[0]
            light = cond.split("_")[1]
            light = light + "_" + cond.split("_")[2]
            
            
        inter = ses.intervalDict[navPathType+"_"+light]
        
        
        sSes.ap.set_intervals(inter)
       
        if onlyArena:
            # this should come after setting the time intervals, will be reset when calling sSes.ap.set_intervals again
            sSes.ap.invalid_outside_spatial_area(environment_shape="circle",radius=43.0,center=np.array([0,0])) ## remove the bridge and homebase

        
        
        for n in sSes.cg.neuron_list:
            n.spike_train.set_intervals(inter)
            n.spatial_properties.firing_rate_map_2d(cm_per_bin =3, smoothing_sigma_cm = 5, smoothing=True,xy_range=xy_range)
            
            
        # calculate map similarity for all pairs of neurons
        sSes.cg.make_pairs(pair_type="combinations")
        rs = np.empty(len(sSes.cg.pairs)) # to store the results
        for i, (j,k) in enumerate(sSes.cg.pairs):
            n1=sSes.cg.neuron_list[j]
            n2=sSes.cg.neuron_list[k]
            rs[i] = map_cor(n1.spatial_properties.firing_rate_map,n2.spatial_properties.firing_rate_map)
        res[cond]=rs
            
    #create a DataFrame from the dictionary
    res = pd.DataFrame(res)
    res = res.add_prefix(prefix)
    res["id1"] = [ sSes.name+"_"+sSes.cg.neuron_list[i].name for i,j in sSes.cg.pairs]
    res["id2"] = [ sSes.name+"_"+sSes.cg.neuron_list[j].name for i,j in sSes.cg.pairs]
    
    res["session"] = sSes.name
    return res

In [11]:
ses,sSes = list(zip(myProject.sessionList,sSesList))[0]
res = mapSimilarityPerCondition(ses,sSes)

In [12]:
res_map=pd.concat([mapSimilarityPerCondition(ses,sSes) for ses,sSes in tqdm(zip(myProject.sessionList,sSesList))],ignore_index=True)

39it [04:39,  7.17s/it]


In [13]:
res_map.shape

(22865, 15)

## Check data integrity of ifr and mapSim df and merge

In [14]:
print(res_map.shape,res_ifr.shape)
if res_map.shape != res_ifr.shape:
    print("problem with the shape of the results")
else:
    print("shape of 2 dfs is matching")

(22865, 15) (22865, 15)
shape of 2 dfs is matching


# Save the data

In [15]:
res = pd.concat([res_ifr,res_map.iloc[:,0:12]],axis=1)
fn=myProject.dataPath+"/results/searchHomingPairs_ifrAsso_mapSim.csv"
print("Saving to",fn)
res.to_csv(fn,index=False)

Saving to /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/results/searchHomingPairs_ifrAsso_mapSim.csv


In [16]:
print('we are here!')

we are here!


## Firing rate map similarities in zone

As a complementary analysis to the IFR association, we can also look at firing rate map similarity in different conditions. 

We use maps that are limited to the zone between the bridge and the center of the arena.


In [17]:
def mapSimilarityPerConditionInZone(ses,sSes, onlyArena=True,
                              conditions=['searchToLeverPath_light','homingFromLeavingLever_light','searchToLeverPath_dark','homingFromLeavingLever_dark',
                                          'searchToLeverPath_light_1', 'searchToLeverPath_light_2', 'searchToLeverPath_dark_1', 'searchToLeverPath_dark_2',
                                          'homingFromLeavingLever_light_1', 'homingFromLeavingLever_light_2', 'homingFromLeavingLever_dark_1', 'homingFromLeavingLever_dark_2'],
                              prefix="mapSimInZone_"):
    """
    Calculate the map similarity of pairs of neurons in a sets of conditions
    
    Arguments: 
    ses: autopipy session
    sSes: spikeA session
    conditions: list of condition names you want to analyze
    prefix: prefix for the DataFrame column names
    
    Return:
    Pandas DataFrame with map similarity for each condition. One row per pair
    """ 
    # we will store the results in a dictionary, one list of scores per condition
    
    
    res={}
    xy_range=np.array([[-10,-40],[10,0]])
    
    for cond in conditions:
        # set the intervals for each neuron and animal pose and calculate the maps
        
        nInfo = len(cond.split("_"))
        if nInfo == 2:
            navPathType = cond.split("_")[0]
            light = cond.split("_")[1]
        if nInfo == 3:
            navPathType = cond.split("_")[0]
            light = cond.split("_")[1]
            light = light + "_" + cond.split("_")[2]
            
            
        inter = ses.intervalDict[navPathType+"_"+light]
        
        
        sSes.ap.set_intervals(inter)
       
        center = [0,-20]
        length = [20,40]
        sSes.ap.invalid_outside_spatial_area(environment_shape="rectangle",center=center,length=length)
        
        
        for n in sSes.cg.neuron_list:
            n.spike_train.set_intervals(inter)
            n.spatial_properties.firing_rate_map_2d(cm_per_bin =2.5, smoothing_sigma_cm = 5, smoothing=True,xy_range=xy_range)
            
            
        # calculate map similarity for all pairs of neurons
        sSes.cg.make_pairs(pair_type="combinations")
        rs = np.empty(len(sSes.cg.pairs)) # to store the results
        for i, (j,k) in enumerate(sSes.cg.pairs):
            n1=sSes.cg.neuron_list[j]
            n2=sSes.cg.neuron_list[k]
            rs[i] = map_cor(n1.spatial_properties.firing_rate_map,n2.spatial_properties.firing_rate_map)
        res[cond]=rs
            
    #create a DataFrame from the dictionary
    res = pd.DataFrame(res)
    res = res.add_prefix(prefix)
    res["id1"] = [ sSes.name+"_"+sSes.cg.neuron_list[i].name for i,j in sSes.cg.pairs]
    res["id2"] = [ sSes.name+"_"+sSes.cg.neuron_list[j].name for i,j in sSes.cg.pairs]
    
    res["session"] = sSes.name
    return res

In [18]:
ses,sSes = list(zip(myProject.sessionList,sSesList))[0]
res = mapSimilarityPerConditionInZone(ses,sSes)

In [19]:
res_map_in_zone=pd.concat([mapSimilarityPerConditionInZone(ses,sSes) for ses,sSes in tqdm(zip(myProject.sessionList,sSesList))],ignore_index=True)

39it [04:28,  6.89s/it]


In [20]:
print(res_map.shape,res_ifr.shape,res_map_in_zone.shape)
if res_map.shape != res_ifr.shape or res_map.shape != res_map_in_zone.shape: 
    print("problem with the shape of the results")
else:
    print("shape of 3 dfs is matching")


(22865, 15) (22865, 15) (22865, 15)
shape of 3 dfs is matching


In [21]:
res = pd.concat([res_ifr,res_map.iloc[:,0:12],res_map_in_zone.iloc[:,0:12]],axis=1)

In [22]:
res.columns

Index(['ifrAsso_searchToLeverPath_light',
       'ifrAsso_homingFromLeavingLever_light',
       'ifrAsso_searchToLeverPath_dark', 'ifrAsso_homingFromLeavingLever_dark',
       'ifrAsso_searchToLeverPath_light_1',
       'ifrAsso_searchToLeverPath_light_2', 'ifrAsso_searchToLeverPath_dark_1',
       'ifrAsso_searchToLeverPath_dark_2',
       'ifrAsso_homingFromLeavingLever_light_1',
       'ifrAsso_homingFromLeavingLever_light_2',
       'ifrAsso_homingFromLeavingLever_dark_1',
       'ifrAsso_homingFromLeavingLever_dark_2', 'id1', 'id2', 'session',
       'mapSim_searchToLeverPath_light', 'mapSim_homingFromLeavingLever_light',
       'mapSim_searchToLeverPath_dark', 'mapSim_homingFromLeavingLever_dark',
       'mapSim_searchToLeverPath_light_1', 'mapSim_searchToLeverPath_light_2',
       'mapSim_searchToLeverPath_dark_1', 'mapSim_searchToLeverPath_dark_2',
       'mapSim_homingFromLeavingLever_light_1',
       'mapSim_homingFromLeavingLever_light_2',
       'mapSim_homingFromLeavingLev

In [23]:
fn=myProject.dataPath+"/results/searchHomingPairs_ifrAsso_mapSim.csv"
print("Saving to",fn)
res.to_csv(fn,index=False)

Saving to /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/results/searchHomingPairs_ifrAsso_mapSim.csv


In [24]:
print('yay!')

yay!
